### 用这个，这个简单

In [ ]:
import torch

Q_BLOCK_SIZE = 3
KV_BLOCK_SIZE = 4
NEG_INF = 1e-10
EPSILON = 1e-10
Q_LEN = 6
K_LEN = 8
# 注意这个和flash attention 1 的顺序相反
Tr = Q_LEN // Q_BLOCK_SIZE      # 外层循环
Tc = K_LEN // KV_BLOCK_SIZE     # 内层循环

Q = torch.randn(1, 1, Q_LEN, 4, requires_grad = True).to(device='cpu')
K = torch.randn(1, 1, K_LEN, 4, requires_grad = True).to(device='cpu')
V = torch.randn(1, 1, K_LEN, 4, requires_grad = True).to(device='cpu')
O = torch.zeros_like(Q, requires_grad = True)           # O[i]：最终输出向量（对应第 i 个 query token）
l = torch.zeros(Q.shape[:-1]).unsqueeze(-1)             # l[i]：当前累积的 softmax 分母
m = torch.ones(Q.shape[:-1]).unsqueeze(-1) * NEG_INF    # m[i]：当前累积的最大 attention score

# 竖着切
Q_blocks = torch.split(Q, Q_BLOCK_SIZE, dim = 2)
K_blocks = torch.split(K, KV_BLOCK_SIZE, dim = 2)
V_blocks = torch.split(V, KV_BLOCK_SIZE, dim = 2)
O_blocks = list(torch.split(O, Q_BLOCK_SIZE, dim=2))
l_blocks = list(torch.split(l, Q_BLOCK_SIZE, dim=2))
m_blocks = list(torch.split(m, Q_BLOCK_SIZE, dim=2))

# start with Q
for i in range(Tr):
    Qi = Q_blocks[i]          # 当前 query block
    Oi = O_blocks[i]
    li = l_blocks[i]
    mi = m_blocks[i]

    for j in range(Tc):
        # if j > i:
        #     continue    # 忽略mask
        Kj = K_blocks[j]
        Vj = V_blocks[j]

        Sij = Qi @ Kj.transpose(-1, -2)
        m_block_ij, _ = torch.max(Sij, dim=-1, keepdim=True)
        mi_new = torch.maximum(mi, m_block_ij)
        P_ij_hat = torch.exp(Sij - mi_new)
        l_block_ij = torch.sum(P_ij_hat, dim=-1, keepdim=True) + EPSILON

        li_new = torch.exp(mi - mi_new) * li + l_block_ij
        Oi = torch.exp(mi - mi_new) * Oi + P_ij_hat @ Vj

        li = li_new
        mi = mi_new
        print(f'-----------O{i} = attn( Q{i}, KV[{j}])---------')
        print(Oi)
    # 这个相当于之前每次内部循环都要除一个li_new，但这次只最后除
    O_blocks[i] = Oi / li_new   # 最后做scaled
    l_blocks[i] = li_new
    m_blocks[i] = mi_new


O = torch.cat(O_blocks, dim=2)
l = torch.cat(l_blocks, dim=2)
m = torch.cat(m_blocks, dim=2)

print("O:", O)



-----------O0 = attn( Q0, KV[0])---------
tensor([[[[ 1.3982e+00,  7.5875e-01,  5.6159e-01,  9.9421e-05],
          [ 8.9726e-01,  6.7523e-01,  6.1780e-01, -6.7770e-01],
          [ 2.3564e+00,  1.0980e+00,  9.9736e-01, -1.4894e-01]]]],
       grad_fn=<AddBackward0>)
-----------O0 = attn( Q0, KV[1])---------
tensor([[[[ 1.9457,  2.4848, -0.3509, -4.2035],
          [ 1.2030, -0.7594,  0.5988, -0.2114],
          [ 0.0331,  1.4902, -0.0575, -2.3302]]]], grad_fn=<AddBackward0>)
-----------O1 = attn( Q1, KV[0])---------
tensor([[[[1.7440, 0.5494, 0.4743, 0.5904],
          [0.6914, 0.2655, 0.1599, 0.5026],
          [2.0992, 0.7122, 0.7146, 0.4859]]]], grad_fn=<AddBackward0>)
-----------O1 = attn( Q1, KV[1])---------
tensor([[[[ 1.8868,  0.7840,  0.4164,  0.4586],
          [ 1.8137, -0.1804, -0.2169, -0.1105],
          [ 2.9275,  1.8271,  0.3610, -0.6856]]]], grad_fn=<AddBackward0>)
O: tensor([[[[ 0.5985,  0.7643, -0.1079, -1.2929],
          [ 0.4139, -0.2612,  0.2060, -0.0727],
      